In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [2]:
housing_df = pd.read_csv('Housing Resale Dataset.csv', parse_dates=['Resale Date'])
housing_df.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Address,Flat Model,Flat Type,Floor Area (sqm),Resale Date,Remaining Lease Duration (Years),Resale Price,Storey Range,Subzone,Postal,Latitude,Longitude,Nearest Dist (MRT),Nearest Dist (SM),Nearest MRT Name,Nearest SM Name,Time to SM,Time to MRT,Time to CBD
0,1.0,100 LOR 1 TOA PAYOH,Improved,3 ROOM,68.0,2017-05-01,52,325000.0,13 TO 15,TOA PAYOH,310100,1.339564,103.848586,0.222592,0.832673,Braddell MRT Station,HDB Hub,897.0,271.0,2210.0
1,2.0,100 LOR 1 TOA PAYOH,Improved,3 ROOM,78.0,2017-05-01,52,330800.0,13 TO 15,TOA PAYOH,310100,1.339564,103.848586,0.222592,0.832673,Braddell MRT Station,HDB Hub,897.0,271.0,2210.0
2,3.0,98 LOR 1 TOA PAYOH,Improved,3 ROOM,78.0,2017-06-01,52,338000.0,13 TO 15,TOA PAYOH,310098,1.339176,103.848380,0.226729,0.786135,Braddell MRT Station,HDB Hub,843.0,312.0,1866.0


In [3]:
# Drop null values in the ds, there were some locations the OneMap API failed to recognise
cols = ['Floor Area (sqm)','Time to MRT','Time to SM','Time to CBD','Remaining Lease Duration (Years)']
housing_df[cols] = housing_df[housing_df[cols] > 0][cols]
housing_df = housing_df.dropna()

In [4]:
# getting only the years 2012 to 2019 since those are the most relevant. 2020 will be used for prediction
start_date = '2012-01-01'
end_date = '2018-12-31'

mask = (housing_df['Resale Date'] > start_date) & (housing_df['Resale Date'] <= end_date)

housing_df = housing_df.loc[mask]
housing_df['Resale Date'].dt.year.unique()

array([2017, 2018, 2012, 2013, 2014, 2016, 2015], dtype=int64)

In [5]:
# y will be our target, the resale price of the property we want to predict
y = housing_df[['Resale Price']]
y.head()

,Resale Price
0,325000.0
1,330800.0
2,338000.0
3,315000.0
4,305000.0


In [6]:
# X contains all the features I plan to use to predict the resale price
X = housing_df[['Floor Area (sqm)','Time to MRT','Time to SM','Time to CBD','Remaining Lease Duration (Years)']]
X.head()

,Floor Area (sqm),Time to MRT,Time to SM,Time to CBD,Remaining Lease Duration (Years)
0,68.0,271.0,897.0,2210.0,52
1,78.0,271.0,897.0,2210.0,52
2,78.0,312.0,843.0,1866.0,52
3,63.0,56.0,882.0,2210.0,52
4,67.0,290.0,637.0,2210.0,52


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [8]:
# check shape to see if it matches
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(57698, 5) (57698, 1)
(14425, 5) (14425, 1)


In [20]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth=5, min_samples_split = 2, learning_rate=0.1, 
                                          loss = 'ls')

In [21]:
clf.fit(X_train, y_train)

C:\Users\Fleming Siow\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingRegressor(max_depth=5, n_estimators=400)

In [22]:
pred = clf.predict(X_test)
pred

array([342270.23191751, 336426.00073997, 627039.64794972, ...,
       292552.32464151, 363922.67601286, 611505.80363767])

In [23]:
clf.score(X_test,y_test)

0.9251707114606229

In [24]:
pred_df = pd.read_csv('Prediction.csv', parse_dates=['Resale Date'])
cols = ['Floor Area (sqm)','Time to MRT','Time to SM','Time to CBD','Remaining Lease Duration (Years)']
pred_df[cols] = pred_df[pred_df[cols] > 0][cols]
pred_df = pred_df.dropna()

X = housing_df[['Floor Area (sqm)','Time to MRT','Time to SM','Time to CBD','Remaining Lease Duration (Years)']]
pred = clf.predict(X)

pred_result = pd.DataFrame(index=np.arange(len(pred)))

for idx, val in enumerate(pred):
    try:
        pred_result.loc[pred_result.index[idx], 'house_id'] = pred_df.iloc[idx]['ID']
        pred_result.loc[pred_result.index[idx], 'predicted_prices'] = val
    except:
        pass

pred_result.to_csv('pred_results.csv', index=False, header=True)

In [80]:
import tabpy_client
from tabpy.tabpy_tools.client import Client

def housing_resale_predictor( _arg1, _arg2, _arg3, _arg4,_arg5, _arg6):
    import pandas as pd

    # Map data in a dictionary
    row = {'area': _arg1,
           'time2mrt': _arg2,
           'time2sm': _arg3,
           'time2cbd': _arg4,
           'lease_duration': _arg5 }
    
    # Convert it into a dataframe
    housing_data = pd.DataFrame(data = row, index=[0])

    # predicting
    pred = clf.predict(housing_data)
    return [resale_price[1] for resale_price in pred]

client = tabpy_client.Client('http://localhost:9004/')
client.deploy('housing_resale_predictor', housing_resale_predictor,"Predicts the resale prices of Singapore's residential\
                properties", override = True)

ResponseError: (400) Error when changing TabPy state: target must be a string. {}